# Azure AI Search dengan Integrasi NVIDIA NIM dan LlamaIndex

Dalam notebook ini, kita akan menunjukkan cara memanfaatkan model AI dari NVIDIA dan LlamaIndex untuk membuat pipeline Retrieval-Augmented Generation (RAG) yang kuat. Kita akan menggunakan LLM dan embedding dari NVIDIA, mengintegrasikannya dengan Azure AI Search sebagai penyimpanan vektor, dan melakukan RAG untuk meningkatkan kualitas dan efisiensi pencarian.

## Manfaat
- **Skalabilitas**: Manfaatkan model bahasa besar dari NVIDIA dan Azure AI Search untuk pencarian yang skalabel dan efisien.
- **Efisiensi Biaya**: Optimalkan pencarian dan pengambilan data dengan penyimpanan vektor yang efisien dan teknik pencarian hibrida.
- **Performa Tinggi**: Gabungkan LLM yang kuat dengan pencarian tervektorisasi untuk respons yang lebih cepat dan akurat.
- **Kualitas**: Pertahankan kualitas pencarian yang tinggi dengan mendasarkan respons LLM pada dokumen yang relevan.

## Prasyarat
- 🐍 Python 3.9 atau lebih tinggi
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 Kunci API NVIDIA untuk akses ke LLM dan Embedding NVIDIA melalui layanan mikro NVIDIA NIM

## Fitur yang Dicakup
- ✅ Integrasi LLM NVIDIA (kita akan menggunakan [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embedding NVIDIA (kita akan menggunakan [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Mode Pengambilan Lanjutan Azure AI Search
- ✅ Pengindeksan Dokumen dengan LlamaIndex
- ✅ RAG menggunakan Azure AI Search dan LlamaIndex dengan LLM NVIDIA

Mari kita mulai!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Instalasi dan Persyaratan
Buat lingkungan Python menggunakan versi Python >3.10.

## Memulai!


Untuk memulai, Anda memerlukan `NVIDIA_API_KEY` untuk menggunakan model NVIDIA AI Foundation:  
1) Buat akun gratis dengan [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Klik pada model pilihan Anda.  
3) Di bawah Input, pilih tab Python, lalu klik **Get API Key** dan kemudian klik **Generate Key**.  
4) Salin dan simpan kunci yang dihasilkan sebagai NVIDIA_API_KEY. Dari sana, Anda seharusnya memiliki akses ke endpoint.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Contoh RAG menggunakan LLM dan Embedding
### 1) Inisialisasi LLM
`llama-index-llms-nvidia`, juga dikenal sebagai konektor LLM NVIDIA, memungkinkan Anda untuk terhubung dan menghasilkan dari model yang kompatibel yang tersedia di katalog API NVIDIA. Lihat di sini untuk daftar model penyelesaian percakapan: https://build.nvidia.com/search?term=Text-to-Text

Di sini kita akan menggunakan **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inisialisasi Embedding
`llama-index-embeddings-nvidia`, juga dikenal sebagai konektor Embeddings NVIDIA, memungkinkan Anda untuk terhubung dan menghasilkan dari model yang kompatibel yang tersedia di katalog API NVIDIA. Kami memilih `nvidia/nv-embedqa-e5-v5` sebagai model embedding. Lihat di sini untuk daftar model embedding teks: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Membuat Mesin Query untuk Mengajukan Pertanyaan pada Data Anda

Berikut adalah sebuah query menggunakan pencarian vektor murni di Azure AI Search dan menghubungkan respons ke LLM kami (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Berikut adalah kueri menggunakan pencarian hybrid di Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analisis Pencarian Vektor
Respons LLM secara akurat menangkap konsekuensi ekonomi utama yang disebutkan dalam teks sumber terkait pasar saham Rusia. Secara khusus, disebutkan bahwa pasar saham Rusia mengalami penurunan signifikan, kehilangan 40% nilainya, dan perdagangan dihentikan akibat situasi yang sedang berlangsung. Respons ini selaras dengan informasi yang diberikan dalam sumber, menunjukkan bahwa LLM berhasil mengidentifikasi dan merangkum detail relevan mengenai dampak pasar saham akibat tindakan Rusia dan sanksi yang diberlakukan.

#### Komentar Node Sumber
Node sumber memberikan penjelasan rinci tentang konsekuensi ekonomi yang dihadapi Rusia akibat sanksi internasional. Teks tersebut menyoroti bahwa pasar saham Rusia kehilangan 40% nilainya, dan perdagangan dihentikan. Selain itu, disebutkan dampak ekonomi lainnya, seperti devaluasi Rubel dan isolasi yang lebih luas terhadap ekonomi Rusia. Respons LLM secara efektif merangkum poin-poin penting dari node ini, dengan fokus pada dampak pasar saham sesuai permintaan kueri.


Sekarang, mari kita lihat sebuah pertanyaan di mana Pencarian Hybrid tidak memberikan jawaban yang kuat:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Pencarian Hibrida: Analisis Respons LLM
Respons LLM dalam contoh Pencarian Hibrida menunjukkan bahwa konteks yang diberikan tidak menentukan tanggal pasti invasi Rusia ke Ukraina. Respons ini mengindikasikan bahwa LLM memanfaatkan informasi yang tersedia dalam dokumen sumber tetapi mengakui ketiadaan detail spesifik dalam teks.

Respons tersebut akurat dalam mengidentifikasi bahwa konteks menyebutkan peristiwa terkait agresi Rusia tetapi tidak menetapkan tanggal invasi secara spesifik. Hal ini menunjukkan kemampuan LLM untuk memahami informasi yang diberikan sambil mengenali kekurangan dalam konten. LLM secara efektif mendorong pengguna untuk mencari sumber eksternal atau catatan sejarah untuk mendapatkan tanggal yang tepat, menunjukkan tingkat kehati-hatian saat informasi tidak lengkap.

### Analisis Node Sumber
Node sumber dalam contoh Pencarian Hibrida berisi kutipan dari pidato yang membahas respons AS terhadap tindakan Rusia di Ukraina. Node-node ini menekankan dampak geopolitik yang lebih luas dan langkah-langkah yang diambil oleh AS dan sekutunya sebagai respons terhadap invasi, tetapi tidak menyebutkan tanggal invasi secara spesifik. Hal ini sejalan dengan respons LLM, yang dengan tepat mengidentifikasi bahwa konteks tidak memiliki informasi tanggal yang spesifik.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: Analisis Respons LLM
Dalam contoh Hybrid w/Reranking, respons LLM memberikan konteks tambahan dengan mencatat bahwa peristiwa tersebut terjadi enam hari sebelum pidato diberikan. Hal ini menunjukkan bahwa LLM mampu menyimpulkan tanggal invasi berdasarkan waktu pidato, meskipun masih memerlukan tanggal pasti pidato untuk akurasi.

Respons ini menunjukkan kemampuan yang lebih baik dalam menggunakan petunjuk konteks untuk memberikan jawaban yang lebih informatif. Ini menyoroti keunggulan reranking, di mana LLM dapat mengakses dan memprioritaskan informasi yang lebih relevan untuk memberikan perkiraan yang lebih mendekati detail yang diinginkan (yaitu, tanggal invasi).

### Analisis Node Sumber
Node sumber dalam contoh ini mencakup referensi tentang waktu invasi Rusia, yang secara spesifik menyebutkan bahwa itu terjadi enam hari sebelum pidato. Meskipun tanggal pasti masih belum dinyatakan secara eksplisit, node tersebut memberikan konteks temporal yang memungkinkan LLM memberikan respons yang lebih bernuansa. Penyertaan detail ini menunjukkan bagaimana reranking dapat meningkatkan kemampuan LLM untuk mengekstrak dan menyimpulkan informasi dari konteks yang diberikan, menghasilkan respons yang lebih akurat dan informatif.


**Catatan:**
Dalam notebook ini, kami menggunakan layanan mikro NVIDIA NIM dari NVIDIA API Catalog.  
API di atas, `NVIDIA (llms)`, `NVIDIAEmbedding`, dan [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
Perlu dicatat, API di atas juga dapat mendukung layanan mikro yang di-host sendiri.

**Contoh:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
